In [149]:
import pandas as pd

df = pd.read_csv(r'C:\Users\ali.karimi\Documents\JupyterProjects\NextPurchaseAmount\online_retail_II.xlsx~\online_retail_merged_clean.csv')


print (df.shape)
display(df.head())


(518596, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [150]:
#Removing invoices without CustomerID
df=df.dropna(subset=['Customer ID'])


#Detecting and removing test customers
import re



def clean_by_customer(df):
    df = df.copy()
    df['Description'] = df['Description'].astype(str).str.lower()
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0.0)

    bad_keywords = [
        'manual', 'test', 'product test', '?', 'sample', 'adjust', 'adjustment',
        'bank charges', 'this is a test product.', 'Manual', 'Bank Charges', 'nan'
    ]
    # escape keywords to make them regex-safe, then join
    escaped = [re.escape(k.lower()) for k in bad_keywords]
    pattern = '|'.join(escaped)

    has_bad_desc = df['Description'].str.contains(pattern, na=False, case=False, regex=True)

    is_zero_price_bad = (df['Price'] == 0) & (~df['Description'].str.contains(re.escape('postage'), na=False, case=False, regex=True))

    bad_customers = df.loc[has_bad_desc | is_zero_price_bad, 'Customer ID'].dropna().unique()

    print("Customers to remove:", len(bad_customers))
    cleaned = df[~df['Customer ID'].isin(bad_customers)].reset_index(drop=True)
    print(f"Removed rows: {len(df) - len(cleaned)} / {len(df)}")

    return cleaned

# استفاده:
clean_df = clean_by_customer(df)

Customers to remove: 660
Removed rows: 147977 / 410763


In [151]:
#Adjusting returns (negative quantities) per customer

from collections import defaultdict
import math

def match_returns_to_purchases_by_customer(
    df,
    date_col='InvoiceDate',
    invoice_col='Invoice',
    customer_col='Customer ID',
    stock_col='StockCode',
    qty_col='Quantity',
    price_col='Price',
    price_tolerance=0.0,        # tolerance as absolute difference (same currency)
    drop_zero_purchases=False   # if True, remove purchase rows that become AdjustedQuantity==0
):
    df = df.copy()
    # ensure types
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df[qty_col] = pd.to_numeric(df[qty_col], errors='coerce').fillna(0).astype(int)
    df[price_col] = pd.to_numeric(df[price_col], errors='coerce').fillna(0.0).round(2)

    # sentinel for NaN customers
    sentinel = '__NO_CUST__'
    df[customer_col] = df[customer_col].fillna(sentinel)

    # sort by customer then date then invoice
    df = df.sort_values([customer_col, date_col, invoice_col]).reset_index(drop=True)

    df['OrigQuantity'] = df[qty_col].astype(int)
    df['AdjustedQuantity'] = df['OrigQuantity'].copy()

    # determine return lines
    is_return = df[invoice_col].astype(str).str.startswith('C', na=False) | (df[qty_col] < 0)

    adjustments = []
    unmatched = []

    # process per-customer
    for cust, group in df.groupby(customer_col, sort=False):
        # ledger: for each stock -> list of purchase entries [{'idx', 'remaining', 'price'}]
        ledger = defaultdict(list)

        # iterate rows in the original order (group preserves ordering because df sorted)
        for i, row in group.iterrows():
            stock = row[stock_col]
            price = float(row[price_col]) if pd.notna(row[price_col]) else 0.0
            q = int(row[qty_col])

            if not is_return.loc[i] and q > 0:
                # purchase: append to ledger
                ledger[stock].append({'idx': i, 'remaining': q, 'price': round(price,2)})
                continue

            if is_return.loc[i]:
                r = abs(q)
                # try to consume from ledger[stock] from the most recent purchases
                while r > 0 and ledger.get(stock):
                    last = ledger[stock][-1]
                    # check price tolerance
                    if abs(last['price'] - round(price,2)) <= price_tolerance:
                        take = min(last['remaining'], r)
                        df.at[last['idx'], 'AdjustedQuantity'] = int(df.at[last['idx'], 'AdjustedQuantity']) - take
                        last['remaining'] -= take
                        r -= take
                        adjustments.append({
                            'return_idx': i,
                            'purchase_idx': last['idx'],
                            'stock': stock,
                            'customer': cust,
                            'qty_adjusted': take,
                            'price_return': round(price,2),
                            'price_purchase': last['price'],
                            'return_invoice': row.get(invoice_col),
                            'purchase_invoice': df.at[last['idx'], invoice_col]
                        })
                        if last['remaining'] == 0:
                            ledger[stock].pop()
                    else:
                        # last purchase price doesn't match within tolerance -> skip it (search earlier)
                        # but we should try earlier purchases (pop to search deeper)
                        # move it aside temporarily
                        # to avoid O(n^2) complexity for huge groups, we scan from tail backward without removing:
                        # We implement a search for the nearest previous purchase that matches price tolerance.
                        found_idx = None
                        for j in range(len(ledger[stock])-1, -1, -1):
                            if abs(ledger[stock][j]['price'] - round(price,2)) <= price_tolerance:
                                found_idx = j
                                break
                        if found_idx is None:
                            # no matching-price purchase for this stock in this customer
                            break
                        # use that purchase
                        last = ledger[stock][found_idx]
                        take = min(last['remaining'], r)
                        df.at[last['idx'], 'AdjustedQuantity'] = int(df.at[last['idx'], 'AdjustedQuantity']) - take
                        last['remaining'] -= take
                        r -= take
                        adjustments.append({
                            'return_idx': i,
                            'purchase_idx': last['idx'],
                            'stock': stock,
                            'customer': cust,
                            'qty_adjusted': take,
                            'price_return': round(price,2),
                            'price_purchase': last['price'],
                            'return_invoice': row.get(invoice_col),
                            'purchase_invoice': df.at[last['idx'], invoice_col]
                        })
                        if last['remaining'] == 0:
                            ledger[stock].pop(found_idx)

                if r > 0:
                    unmatched.append({
                        'return_idx': i,
                        'stock': stock,
                        'customer': cust,
                        'unmatched_qty': r,
                        'price_return': round(price,2),
                        'return_invoice': row.get(invoice_col)
                    })

    adjustments_df = pd.DataFrame(adjustments)
    unmatched_df = pd.DataFrame(unmatched)

    # optionally drop purchases that became zero
    if drop_zero_purchases:
        zero_idxs = df.loc[df['AdjustedQuantity'] == 0].index
        df = df.drop(zero_idxs).reset_index(drop=True)

    # restore NaN customers
    df[customer_col] = df[customer_col].replace({sentinel: pd.NA})

    # summary
    print(f"Return lines: {int(is_return.sum())}")
    print(f"Adjustment rows logged: {len(adjustments_df)}")
    print(f"Unmatched returns: {len(unmatched_df)}")

    return df, adjustments_df, unmatched_df

adjusted_df, adjustments_df, unmatched_df= match_returns_to_purchases_by_customer(clean_df)


Return lines: 5928
Adjustment rows logged: 4816
Unmatched returns: 1232


In [152]:
#removing the invoices which we adjust their effects in other invoices

def remove_fully_handled_returns(df, adjustments_df,
                                 invoice_col='Invoice', qty_col='Quantity',
                                 orig_qty_col='OrigQuantity', adj_qty_col='AdjustedQuantity'):
    """
    df: حاصل تابع اصلی (شامل OrigQuantity و AdjustedQuantity)
    adjustments_df: لاگِ adjustments برگشتی‌ها که حاوی فیلد 'return_idx' و 'qty_adjusted' است
    خروجی: (cleaned_df, dropped_rows_df, updated_df) - cleaned_df: بدون ردیف‌های برگشتی که کامل تسویه شدند
    """
    df = df.copy()
    if adjustments_df.empty:
        print("No adjustments logged — nothing to remove.")
        return df, df.loc[[]].copy(), df

    # مجموع qty_adjusted برای هر return_idx
    matched_by_return = adjustments_df.groupby('return_idx')['qty_adjusted'].sum()

    # نگه‌داری ایندکس‌هایی که کامل تسویه شده‌اند و ایندکس‌هایی که جزئی تسویه شدند
    to_drop = []
    to_update = []

    for ret_idx, matched_qty in matched_by_return.items():
        # ممکنه ایندکس مربوطه در df موجود نباشه (اگر قبلاً اندکس‌ها ریست شده باشن)
        if ret_idx not in df.index:
            continue
        orig = df.at[ret_idx, orig_qty_col]  # منفی
        # matched_qty مثبت است؛ new_adj = orig + matched_qty
        new_adj = int(orig + matched_qty)
        df.at[ret_idx, adj_qty_col] = new_adj
        if new_adj == 0:
            to_drop.append(ret_idx)
        else:
            # اگر هنوز منفی مونده (مثلاً orig=-3 matched=1 -> new_adj=-2)
            to_update.append(ret_idx)

    # حذف ردیف‌هایی که به صفر رسیدند و orig منفی بودن (یعنی برگشتی کامل)
    dropped_rows = df.loc[to_drop].copy() if to_drop else df.loc[[]].copy()
    cleaned = df.drop(index=to_drop).reset_index(drop=True)

    print(f"Dropped fully-handled return rows: {len(to_drop)}")
    print(f"Partially-updated return rows (still negative): {len(to_update)}")

    return cleaned, dropped_rows, df  # df حاوی AdjustedQuantityهای به‌روز شده
# 1) فرض: اجرا کرده‌ای
# adjusted_df, adjustments_df, unmatched_df = match_returns_to_purchases_by_customer(clean_df)

# 2) حالا این تابع را اجرا کن
cleaned_df, dropped_returns_df, updated_df_with_adjusts = remove_fully_handled_returns(
    adjusted_df, adjustments_df,
    invoice_col='Invoice', qty_col='Quantity',
    orig_qty_col='OrigQuantity', adj_qty_col='AdjustedQuantity')

Dropped fully-handled return rows: 4696
Partially-updated return rows (still negative): 50


In [153]:
#Removing returns before the first positive purchase

import pandas as pd

def drop_negative_before_first_positive(df,
                                        customer_col='Customer ID',
                                        date_col='InvoiceDate',
                                        qty_col='OrigQuantity',
                                        return_dropped=False):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    sentinel = '__NO_CUST__'
    df[customer_col] = df[customer_col].fillna(sentinel)
    df = df.sort_values([customer_col, date_col]).reset_index(drop=True)

    drop_idxs = []
    for cust, g in df.groupby(customer_col, sort=False):
        pos = g[g[qty_col] > 0]
        if not pos.empty:
            first_pos_date = pos[date_col].min()
            mask = (g[qty_col] < 0) & (g[date_col] < first_pos_date)
        else:
            # هیچ خرید مثبتی نیست -> حذف همه منفی‌ها برای این مشتری
            mask = (g[qty_col] < 0)
        drop_idxs.extend(g.loc[mask].index.tolist())

    dropped = df.loc[drop_idxs].copy()
    cleaned = df.drop(index=drop_idxs).reset_index(drop=True)
    cleaned[customer_col] = cleaned[customer_col].replace({sentinel: pd.NA})
    dropped[customer_col] = dropped[customer_col].replace({sentinel: pd.NA})

    print(f"Dropped {len(drop_idxs)} rows (negative before first positive) out of {len(df)} rows.")

    if return_dropped:
        return cleaned, dropped
    return cleaned

adjusted_df_clean = drop_negative_before_first_positive(cleaned_df)


Dropped 448 rows (negative before first positive) out of 258090 rows.


In [158]:
#How many negative quantity remained?

df['line_amount'] = df['Price'] * df['Quantity']  # اگر هنوز این ستون نیست
total_line_sales = df['line_amount'].sum()
adj_total = invoice_adj['InvoiceAdjustment'].sum() if 'invoice_adj' in locals() else -3839.59

print(f"Total line sales: {total_line_sales:.2f}")
print(f"Total invoice adjustments (service/discount): {adj_total:.2f}")
print(f"Adjustment / Sales ratio: {adj_total/total_line_sales:.4%}")

Total line sales: 8314855.58
Total invoice adjustments (service/discount): -3839.59
Adjustment / Sales ratio: -0.0462%


In [159]:
#Removing other negative Quantity

adjusted_df_clean=adjusted_df_clean[adjusted_df_clean['AdjustedQuantity']>0]

In [160]:
adjusted_df_clean.to_excel("adjusted_online_retail.xlsx", index=False)
